In [1]:
import numpy as np
import tqdm
import pickle

import tensorflow as tf 
import gc
import os

In [2]:
flow_size=300
negetive_samples=199

all_runs={'8872':'192.168.122.117','8802':'192.168.122.117','8873':'192.168.122.67','8803':'192.168.122.67',
         '8874':'192.168.122.113','8804':'192.168.122.113','8875':'192.168.122.120',
        '8876':'192.168.122.30','8877':'192.168.122.208','8878':'192.168.122.58'}

dataset=[]

for name in all_runs:
    dataset+=pickle.load(open('../dataset/%s_tordata300.pickle'%name))
    
len_tr=len(dataset)
print("Dataset length: ", len_tr)
#train_ratio=float(len_tr-6000)/float(len_tr)
train_ratio=0.8
rr= range(len(dataset))
np.random.shuffle(rr)

train_index=rr[:int(len_tr*train_ratio)]
test_index= rr[int(len_tr*train_ratio):] #range(len(dataset_test)) # #
pickle.dump(test_index,open('test_index300.pickle','w'))


('Dataset length: ', 7324)


In [3]:
print(dataset[0])

{'there': ({'<-': [0.2538440227508545, 0.2879009246826172, 0.179901123046875, 0.4883768558502197, 0.26952195167541504, 0.14073705673217773, 0.14699506759643555, 0.49796295166015625, 0.22753500938415527, 0.524691104888916, 0.11671590805053711, 0.07928991317749023, 0.07904720306396484, 0.13978290557861328, 0.11146306991577148, 0.07183384895324707, 0.08247113227844238, 0.07933902740478516, 0.09266281127929688, 0.18150019645690918, 0.1405048370361328, 0.08318400382995605, 0.07167816162109375, 0.07188296318054199, 0.0953359603881836, 0.14577603340148926, 0.1538529396057129, 0.1884009838104248, 0.14070701599121094, 0.1407639980316162, 0.18001198768615723, 0.19310593605041504, 0.07175207138061523, 0.08289599418640137, 0.07713103294372559, 0.11132407188415527, 0.0993199348449707, 0.10880303382873535, 0.10046792030334473, 0.15383291244506836, 0.14374899864196777, 0.07486701011657715, 0.03496503829956055, 0.08182096481323242, 0.0925741195678711, 0.07768487930297852, 0.07971310615539551, 0.096911

In [32]:
print(dataset[0]["there"])
#prettifyied with online converted tool https://codebeautify.org/python-formatter-beautifier

({'<-': [0.2538440227508545, 0.2879009246826172, 0.179901123046875, 0.4883768558502197, 0.26952195167541504, 0.14073705673217773, 0.14699506759643555, 0.49796295166015625, 0.22753500938415527, 0.524691104888916, 0.11671590805053711, 0.07928991317749023, 0.07904720306396484, 0.13978290557861328, 0.11146306991577148, 0.07183384895324707, 0.08247113227844238, 0.07933902740478516, 0.09266281127929688, 0.18150019645690918, 0.1405048370361328, 0.08318400382995605, 0.07167816162109375, 0.07188296318054199, 0.0953359603881836, 0.14577603340148926, 0.1538529396057129, 0.1884009838104248, 0.14070701599121094, 0.1407639980316162, 0.18001198768615723, 0.19310593605041504, 0.07175207138061523, 0.08289599418640137, 0.07713103294372559, 0.11132407188415527, 0.0993199348449707, 0.10880303382873535, 0.10046792030334473, 0.15383291244506836, 0.14374899864196777, 0.07486701011657715, 0.03496503829956055, 0.08182096481323242, 0.0925741195678711, 0.07768487930297852, 0.07971310615539551, 0.0969119071960449

In [4]:
def generate_data(dataset,train_index,test_index,flow_size):
    
    global negetive_samples

    all_samples=len(train_index)
    labels=np.zeros((all_samples*(negetive_samples+1),1))
    l2s=np.zeros((all_samples*(negetive_samples+1),8,flow_size,1))

    index=0
    random_ordering=[]+train_index
    for i in tqdm.tqdm(train_index):
        #[]#list(lsh.find_k_nearest_neighbors((Y_train[i]/ np.linalg.norm(Y_train[i])).astype(np.float64),(50)))

        l2s[index,0,:,0]=np.array(dataset[i]['here'][0]['<-'][:flow_size])*1000.0
        l2s[index,1,:,0]=np.array(dataset[i]['there'][0]['->'][:flow_size])*1000.0
        l2s[index,2,:,0]=np.array(dataset[i]['there'][0]['<-'][:flow_size])*1000.0
        l2s[index,3,:,0]=np.array(dataset[i]['here'][0]['->'][:flow_size])*1000.0

        l2s[index,4,:,0]=np.array(dataset[i]['here'][1]['<-'][:flow_size])/1000.0
        l2s[index,5,:,0]=np.array(dataset[i]['there'][1]['->'][:flow_size])/1000.0
        l2s[index,6,:,0]=np.array(dataset[i]['there'][1]['<-'][:flow_size])/1000.0
        l2s[index,7,:,0]=np.array(dataset[i]['here'][1]['->'][:flow_size])/1000.0


        if index % (negetive_samples+1) !=0:
            print index , len(nears)
            raise
        labels[index,0]=1
        m=0
        index+=1
        np.random.shuffle(random_ordering)
        for idx in random_ordering:
            if idx==i or m>(negetive_samples-1):
                continue

            m+=1

            l2s[index,0,:,0]=np.array(dataset[idx]['here'][0]['<-'][:flow_size])*1000.0
            l2s[index,1,:,0]=np.array(dataset[i]['there'][0]['->'][:flow_size])*1000.0
            l2s[index,2,:,0]=np.array(dataset[i]['there'][0]['<-'][:flow_size])*1000.0
            l2s[index,3,:,0]=np.array(dataset[idx]['here'][0]['->'][:flow_size])*1000.0

            l2s[index,4,:,0]=np.array(dataset[idx]['here'][1]['<-'][:flow_size])/1000.0
            l2s[index,5,:,0]=np.array(dataset[i]['there'][1]['->'][:flow_size])/1000.0
            l2s[index,6,:,0]=np.array(dataset[i]['there'][1]['<-'][:flow_size])/1000.0
            l2s[index,7,:,0]=np.array(dataset[idx]['here'][1]['->'][:flow_size])/1000.0

            #l2s[index,0,:,0]=Y_train[i]#np.concatenate((Y_train[i],X_train[idx]))#(Y_train[i]*X_train[idx])/(np.linalg.norm(Y_train[i])*np.linalg.norm(X_train[idx]))
            #l2s[index,1,:,0]=X_train[idx]



            labels[index,0]=0
            index+=1




    #lsh.setup((X_test / np.linalg.norm(X_test,axis=1,keepdims=True)) .astype(np.float64))
    index_hard=0
    num_hard_test=0
    l2s_test=np.zeros((len(test_index)*(negetive_samples+1),8,flow_size,1))
    labels_test=np.zeros((len(test_index)*(negetive_samples+1)))
    l2s_test_hard=np.zeros((num_hard_test*num_hard_test,2,flow_size,1))
    index=0
    random_test=[]+test_index

    for i in tqdm.tqdm(test_index):
        #list(lsh.find_k_nearest_neighbors((Y_test[i]/ np.linalg.norm(Y_test[i])).astype(np.float64),(50)))



        if index % (negetive_samples+1) !=0:
            print index, nears
            raise 
        m=0

        np.random.shuffle(random_test)
        for idx in random_test:
            if idx==i or m>(negetive_samples-1):
                continue

            m+=1
            l2s_test[index,0,:,0]=np.array(dataset[idx]['here'][0]['<-'][:flow_size])*1000.0
            l2s_test[index,1,:,0]=np.array(dataset[i]['there'][0]['->'][:flow_size])*1000.0
            l2s_test[index,2,:,0]=np.array(dataset[i]['there'][0]['<-'][:flow_size])*1000.0
            l2s_test[index,3,:,0]=np.array(dataset[idx]['here'][0]['->'][:flow_size])*1000.0

            l2s_test[index,4,:,0]=np.array(dataset[idx]['here'][1]['<-'][:flow_size])/1000.0
            l2s_test[index,5,:,0]=np.array(dataset[i]['there'][1]['->'][:flow_size])/1000.0
            l2s_test[index,6,:,0]=np.array(dataset[i]['there'][1]['<-'][:flow_size])/1000.0
            l2s_test[index,7,:,0]=np.array(dataset[idx]['here'][1]['->'][:flow_size])/1000.0
            labels_test[index]=0
            index+=1

        l2s_test[index,0,:,0]=np.array(dataset[i]['here'][0]['<-'][:flow_size])*1000.0
        l2s_test[index,1,:,0]=np.array(dataset[i]['there'][0]['->'][:flow_size])*1000.0
        l2s_test[index,2,:,0]=np.array(dataset[i]['there'][0]['<-'][:flow_size])*1000.0
        l2s_test[index,3,:,0]=np.array(dataset[i]['here'][0]['->'][:flow_size])*1000.0

        l2s_test[index,4,:,0]=np.array(dataset[i]['here'][1]['<-'][:flow_size])/1000.0
        l2s_test[index,5,:,0]=np.array(dataset[i]['there'][1]['->'][:flow_size])/1000.0
        l2s_test[index,6,:,0]=np.array(dataset[i]['there'][1]['<-'][:flow_size])/1000.0
        l2s_test[index,7,:,0]=np.array(dataset[i]['here'][1]['->'][:flow_size])/1000.0
        #l2s_test[index,2,:,0]=dataset[i]['there'][0]['->'][:flow_size]
        #l2s_test[index,3,:,0]=dataset[i]['here'][0]['<-'][:flow_size]

        #l2s_test[index,0,:,1]=dataset[i]['here'][1]['->'][:flow_size]
        #l2s_test[index,1,:,1]=dataset[i]['there'][1]['<-'][:flow_size]
        #l2s_test[index,2,:,1]=dataset[i]['there'][1]['->'][:flow_size]
        #l2s_test[index,3,:,1]=dataset[i]['here'][1]['<-'][:flow_size]
        labels_test[index]=1

        index+=1
    return l2s, labels,l2s_test,labels_test

In [5]:
l2s, labels, l2s_test, labels_test = generate_data(dataset=dataset, train_index=train_index, test_index=test_index, flow_size=flow_size)


100%|██████████| 1465/1465 [00:34<00:00, 42.96it/s]


In [44]:
# Finding out What 0 and 1 is in the generate_data function for [here/there[0/1]
print(np.array(dataset[0]['there'][0]['<-'][:flow_size]))

print(np.array(dataset[0]['here'][1]['<-'][:flow_size]))

[2.53844023e-01 2.87900925e-01 1.79901123e-01 4.88376856e-01
 2.69521952e-01 1.40737057e-01 1.46995068e-01 4.97962952e-01
 2.27535009e-01 5.24691105e-01 1.16715908e-01 7.92899132e-02
 7.90472031e-02 1.39782906e-01 1.11463070e-01 7.18338490e-02
 8.24711323e-02 7.93390274e-02 9.26628113e-02 1.81500196e-01
 1.40504837e-01 8.31840038e-02 7.16781616e-02 7.18829632e-02
 9.53359604e-02 1.45776033e-01 1.53852940e-01 1.88400984e-01
 1.40707016e-01 1.40763998e-01 1.80011988e-01 1.93105936e-01
 7.17520714e-02 8.28959942e-02 7.71310329e-02 1.11324072e-01
 9.93199348e-02 1.08803034e-01 1.00467920e-01 1.53832912e-01
 1.43748999e-01 7.48670101e-02 3.49650383e-02 8.18209648e-02
 9.25741196e-02 7.76848793e-02 7.97131062e-02 9.69119072e-02
 1.18772030e-01 7.92281628e-02 1.09428883e-01 1.31968021e-01
 7.94448853e-02 2.21710205e-01 1.80373907e-01 1.40376091e-01
 1.40492916e-01 8.01570415e-02 3.90008926e-01 1.05734110e-01
 5.50293922e-03 8.45389366e-02 1.02419138e-01 3.01980972e-01
 1.11060858e-01 7.169604

In [10]:
print(l2s.shape)

(1171800, 8, 300, 1)


In [11]:
print(l2s[0].shape)

(8, 300, 1)


In [35]:
print(labels.shape)

(1171800, 1)


In [36]:
print(l2s_test.shape)

(293000, 8, 300, 1)
